In [16]:
import scipy.stats as ss
import numpy as np


# ### 1. Проверка гипотезы однородности

#  Критерий пустых блоков

n = 4000
m = 8000
rho = m / n
alpha = 1
beta = 1.2
M = n / (1 + rho)
sigma = n * rho**2 / (1 + rho)**3

X = ss.expon.rvs(size=n, scale=alpha)
Y = ss.expon.rvs(size=m, scale=beta)
X = np.append(X, [-1000, 1000]) # add -inf +inf
X = np.sort(X)
Y = np.sort(Y)

intervals = range(0, n + 1)

for i in range(1, len(X)):
    count = 0
    for j in Y:
        if (j < X[i]) & (j >= X[i - 1]):
            count += 1
        intervals[i - 1] = count

count_zero = intervals.count(0)
critical = M + ss.norm.ppf(0.95) * np.sqrt(sigma)
print(count_zero)
if count_zero > critical:
    print 'Выборки X и Y - неоднородны'
else:
    print 'Выборки X и Y - однородны'

1386
Выборки X и Y - неоднородны


In [40]:
print 'Critical value:'
print ss.norm.ppf(0.95) * np.sqrt(sigma) + M

Critical value:
1373.02101604


In [20]:
# <b>Б.</b> Критерий Хи квадрат для проверки Х ~ Pois(2)


k = 3  # k в Пуассоне
r = 5  # разбиваем до r-1

n = [200, 500, 800] # 3 series 

X = ss.poisson.rvs(2, size=n[0])
Y = ss.poisson.rvs(2, size=n[1])
Z = ss.poisson.rvs(2, size=n[2])

sup = 15
intervals = range(0, sup + 1, sup / r)


def count_blocks(sample, intervals):
    counts = np.zeros(5)
    for i in range(0, len(intervals) - 1):
        count = 0
        for j in sample:
            if(j < intervals[i + 1]) & (j >= intervals[i]):
                count = count + 1
        counts[i] = count
    return counts


counts_x = count_blocks(X, intervals)
counts_y = count_blocks(Y, intervals)
counts_z = count_blocks(Z, intervals)
counts = np.vstack((counts_x, counts_y, counts_z))
delta = 0

for i in range(0, k):
    for j in range(0, r):
        if counts.sum(axis=0)[j] > 0:
            delta += counts[i, j]**2 / (n[i] * counts.sum(axis=0)[j])
            
delta = (delta - 1) * sum(n)

if delta > ss.chi2.ppf(0.95, df=(k - 1) * (r - 1)):
    print 'Выборки X и Y - неоднородны'
else:
    print 'Выборки X и Y - однородны'

Выборки X и Y - однородны


In [36]:
print delta 
print 'Critical value:'
print ss.chi2.ppf(0.95, df=(k - 1) * (r - 1))

4.84181423084
Critical value:
122.10773461


In [31]:
# ### 2. Проверка гипотезы независимости

# Критерий Хи квадрат

r = 8
k = 15

n = 1000
X = ss.poisson.rvs(2, size=n)
Y = ss.poisson.rvs(5, size=n)

sup = 20
intervals_x = range(0, sup + 1, sup / r)
intervals_y = range(0, sup + 1, sup / k)

counts_ij = np.zeros((r, k))

for l in range(0, r):
    for m in range(0, k):
        count_ij = 0
        for j in range(0, n):
            if (intervals_x[l + 1] > X[j] & intervals_x[l] <= X[j] & intervals_y[m] <= Y[j] & intervals_y[m + 1] > Y[j]):
                count_ij = count_ij + 1
    counts_ij[l, m] = count_ij

counts_i = counts_ij.sum(axis=0)
counts_j = counts_ij.sum(axis=1)

delta_r_k = 0
for i in range(0, len(counts_i)):
    for j in range(0, len(counts_j)):
        if (counts_j[j] > 0) & (counts_i[i] > 0):
            delta_r_k += counts_ij[i, j]**2 / (counts_j[j] * counts_i[i])

delta_r_k = (delta_r_k - 1) * n

if delta_r_k > ss.chi2.ppf(0.95, df=(k - 1) * (r - 1)):
    print "X и Y - зависимы"
else:
    print "X и Y - независимы"

X и Y - независимы


In [35]:
print delta_r_k
print 'Critical value:'
print ss.chi2.ppf(0.95, df=(k - 1) * (r - 1))

-1000
Critical value:
122.10773461


In [21]:
# <b>Б.</b> Критерий Спирмана


n = 200
X = ss.norm.rvs(size=n)
Y = ss.norm.rvs(size=n)
Xs = np.sort(X)
Ys = np.sort(Y)

R = np.zeros(n)  # матрица рангов, для 1й выборки
S = np.zeros(n)  # матрица рангов, для 2й выборки

for i in range(n):
    k = np.where(X == Xs[i])[0][0]
    R[k] = i + 1

    l = np.where(Y == Ys[i])[0][0]
    S[l] = i + 1

rho = 1.0 - sum([(R[i] - S[i])**2 for i in range(n)]) *     6.0 / (n * (n - 1) * (n + 1))

if np.around(rho) == 1:
    print "X и Y - зависимы (строгая прямая линейная зависимость)"
elif np.around(rho) == -1:
    print "X и Y - зависимы (обратная линейная зависимость)"
elif np.around(rho) == 0:
    print "X и Y - независимы"

X и Y - независимы


In [39]:
print np.around(rho)

0.0


In [5]:
# ### 3. Проверка гипотезы случайности

# In[6]:

n = 500
X = ss.uniform.rvs(size=n)

t = 0
for i in range(0, n):
    for j in range(i, n):
        if X[j] < X[i]:
            t = t + 1

print "Количество инверсий: %i" % t

if abs(t - n * (n - 1) / 4) >= ss.norm.ppf(0.95) * n * (n**(0.5)) / 6:
    print "Гипотеза отвергнута"
else:
    print "Гипотеза принята"



Количество инверсий: 60356
Гипотеза принята


In [38]:
print abs(t - n * (n - 1) / 4)
print 'Critical value:'
print ss.norm.ppf(0.95) * n * (n**(0.5)) / 6

189394
Critical value:
8669.13979793
